# Week 5 スライド

## NumPy Data

In [1]:
#---- week05/practice
import numpy as np

rng = np.random.default_rng(100)   # 100 は乱数の種
x = rng.choice(range(100, 500), 15)
np.mean(x)
np.median(x)
np.quantile(x, 0.9)
x > 300
x[x > 300]

array([406, 433, 338, 335, 489, 478, 338, 416, 469])

In [2]:
#---- week05/tf-vector
x > 400
x[x > 400]

array([406, 433, 489, 478, 416, 469])

In [3]:
#---- week05/tf-sum
True + True + False
True + False + False + True + True

3

In [4]:
#---- week05/tf-count
np.sum(x > 400)

6

In [5]:
#---- week05/tf-ratio
np.mean(x > 400)

0.4

In [6]:
#---- week05/median
np.median(x)

338.0

In [7]:
#---- week05/quantile
np.quantile(x, 0.9)    # 下から90% の値

474.4

## 相対的貧困

In [8]:
#---- week05/disposable-income
import pandas as pd
frame = pd.DataFrame({
    'household': list('AAABBCD'),
    'income': [400, 300, 0, 1000, 0, 250, 150.]
}, index=range(1, 8))
frame

,household,income
1,A,400.0
2,A,300.0
3,A,0.0
4,B,1000.0
5,B,0.0
6,C,250.0
7,D,150.0


In [9]:
#---- week05/disposable-income2
income = (frame.groupby('household')
 .transform(lambda x: x.sum() / np.sqrt(x.size)))
income

,income
1,404.145188
2,404.145188
3,404.145188
4,707.106781
5,707.106781
6,250.000000
7,150.000000


In [10]:
#---- week05/less-than-rpl
income < np.median(income) / 2

,income
1,False
2,False
3,False
4,False
5,False
6,False
7,True


In [11]:
#---- week05/relative-poverty
np.mean(income < np.median(income) / 2) * 100

income    14.285714
dtype: float64

## Gini 係数

In [12]:
#---- week05/gini-data-gen
xlow = rng.choice(np.arange(10, 50.), 10)
xhigh = rng.choice(np.arange(80, 200.), 10)
x = np.r_[xlow, xhigh]
x

array([ 46.,  10.,  37.,  38.,  17.,  30.,  49.,  37.,  21.,  22., 155.,
        97., 149., 148., 151., 172., 144., 166., 199.,  95.])

In [13]:
#---- week05/gini-sorted
y = np.sort(x)   # Step 1
y

array([ 10.,  17.,  21.,  22.,  30.,  37.,  37.,  38.,  46.,  49.,  95.,
        97., 144., 148., 149., 151., 155., 166., 172., 199.])

In [14]:
#---- week05/gini-cum-freq
Y = np.r_[0, y.cumsum()] / y.sum()  # Step 2, 3
Y

array([0.        , 0.00560852, 0.01514302, 0.02692092, 0.03925967,
       0.05608525, 0.07683679, 0.09758833, 0.11890073, 0.14469994,
       0.17218172, 0.2254627 , 0.2798654 , 0.36062815, 0.44363432,
       0.52720135, 0.61189007, 0.69882221, 0.79192372, 0.88839035,
       1.        ])

In [15]:
#---- week05/gini-formula
gini = 1 - (2 * np.sum(Y) - 1) / y.size   # Step 4, 5, 6
gini

0.39189568143578235

## Top percentile share

In [16]:
#---- week05/top-percentile-data
x

array([ 46.,  10.,  37.,  38.,  17.,  30.,  49.,  37.,  21.,  22., 155.,
        97., 149., 148., 151., 172., 144., 166., 199.,  95.])

In [17]:
#---- week05/top-percentile-threshold
np.quantile(x, 0.99)

193.86999999999995

In [18]:
#---- week05/top-percentile-share
np.sum(x[x > np.quantile(x, 0.99)]) / np.sum(x)

0.11160964666292765

In [19]:
#---- END

## 参考: Python のループ・ベクトル計算

500000個の正規乱数を発生させて，各要素の絶対値を取る簡単な計算をする。

1. `r_abs1` はループの中で if-else を使って1つずつ処理する。もっとも汎用性が高い。一見すると無駄も少ないように見える。
1. `r_abs2` は `np.where` を使ってベクトル化する。汎用性の高い方法。
1. `r_abs3` は `np.abs` を使った例，効率はよさそうだけど汎用性が低い。

In [20]:
r = rng.normal(size=(500000))
r_abs1 = np.empty_like(r)

#### Case 1

In [21]:
%%timeit
for i in range(r.size):
    rr = r[i]
    if rr >= 0:
        r_abs1[i] = rr
    else:
        r_abs1[i] = -rr

187 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Case 2

In [22]:
%%timeit
r_abs2 = np.where(r >= 0, r, -r)

3.19 ms ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Case 3

In [23]:
%%timeit
r_abs3 = np.abs(r)

620 µs ± 7.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 結果

Case 2 は Case 1 より 100くらいはやい。Case 3 は Case 2 より 8倍くらい早い。一般的な教訓として覚えておこう。

- 特定のケースを処理するために作られた関数は実行速度が早い。必要なループ処理が Fortran/C で書かれている。
- ベクトル計算は無駄な計算をしている（例えば，全要素に対して `-r` を計算している）ようだけど，ループで処理するよりは断然まし。
- 本当にその for/while が必要かどうかを吟味する。